Link to the notebook: [here](https://www.kaggle.com/code/niharika41298/netflix-visualizations-recommendation-eda)

# 1. Introduction 

The goal of this project is to develop a content-based recommendation engine for movies and TV shows on Netflix. I will compare two different methods:

1. Using *cast, director, country, rating and genres* as features.
2. Using the words in the movie/TV show *descriptions* as features.

# 2. Imports

In [2]:
import numpy as np
import pandas as pd
import re

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# 3. Loading data

In [3]:
data = pd.read_csv('../input/netflix-shows/netflix_titles.csv')
data.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,TV Show,3%,NaN,"João Miguel, Bianca Comparato, Michel Gomes, R...",Brazil,"August 14, 2020",2020,TV-MA,4 Seasons,"International TV Shows, TV Dramas, TV Sci-Fi &...",In a future where the elite inhabit an island ...
1,s2,Movie,7:19,Jorge Michel Grau,"Demián Bichir, Héctor Bonilla, Oscar Serrano, ...",Mexico,"December 23, 2016",2016,TV-MA,93 min,"Dramas, International Movies",After a devastating earthquake hits Mexico Cit...
2,s3,Movie,23:59,Gilbert Chan,"Tedd Chan, Stella Chung, Henley Hii, Lawrence ...",Singapore,"December 20, 2018",2011,R,78 min,"Horror Movies, International Movies","When an army recruit is found dead, his fellow..."
3,s4,Movie,9,Shane Acker,"Elijah Wood, John C. Reilly, Jennifer Connelly...",United States,"November 16, 2017",2009,PG-13,80 min,"Action & Adventure, Independent Movies, Sci-Fi...","In a postapocalyptic world, rag-doll robots hi..."
4,s5,Movie,21,Robert Luketic,"Jim Sturgess, Kevin Spacey, Kate Bosworth, Aar...",United States,"January 1, 2020",2008,PG-13,123 min,Dramas,A brilliant group of students become card-coun...


In [4]:
data.groupby('type').count()

,show_id,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
type,,,,,,,,,,,
Movie,5377,5377,5214,4951,5147,5377,5377,5372,5377,5377,5377
TV Show,2410,2410,184,2118,2133,2400,2410,2408,2410,2410,2410


In [5]:
data = data.dropna(subset=['cast', 'country', 'rating'])

# 4. Developing Recommendation Engine using *cast, director, country, rating and genres*

In [6]:
movies = data[data['type'] == 'Movie'].reset_index()
movies = movies.drop(['index', 'show_id', 'type', 'date_added', 'release_year', 'duration', 'description'], axis=1)
movies.head()

,title,director,cast,country,rating,listed_in
0,7:19,Jorge Michel Grau,"Demián Bichir, Héctor Bonilla, Oscar Serrano, ...",Mexico,TV-MA,"Dramas, International Movies"
1,23:59,Gilbert Chan,"Tedd Chan, Stella Chung, Henley Hii, Lawrence ...",Singapore,R,"Horror Movies, International Movies"
2,9,Shane Acker,"Elijah Wood, John C. Reilly, Jennifer Connelly...",United States,PG-13,"Action & Adventure, Independent Movies, Sci-Fi..."
3,21,Robert Luketic,"Jim Sturgess, Kevin Spacey, Kate Bosworth, Aar...",United States,PG-13,Dramas
4,122,Yasir Al Yasiri,"Amina Khalil, Ahmed Dawood, Tarek Lotfy, Ahmed...",Egypt,TV-MA,"Horror Movies, International Movies"


In [7]:
tv = data[data['type'] == 'TV Show'].reset_index()
tv = tv.drop(['index', 'show_id', 'type', 'date_added', 'release_year', 'duration', 'description'], axis=1)
tv.head()

,title,director,cast,country,rating,listed_in
0,3%,NaN,"João Miguel, Bianca Comparato, Michel Gomes, R...",Brazil,TV-MA,"International TV Shows, TV Dramas, TV Sci-Fi &..."
1,46,Serdar Akar,"Erdal Beşikçioğlu, Yasemin Allen, Melis Birkan...",Turkey,TV-MA,"International TV Shows, TV Dramas, TV Mysteries"
2,1983,NaN,"Robert Więckiewicz, Maciej Musiał, Michalina O...","Poland, United States",TV-MA,"Crime TV Shows, International TV Shows, TV Dramas"
3,​SAINT SEIYA: Knights of the Zodiac,NaN,"Bryson Baugus, Emily Neves, Blake Shepard, Pat...",Japan,TV-14,"Anime Series, International TV Shows"
4,#blackAF,NaN,"Kenya Barris, Rashida Jones, Iman Benson, Genn...",United States,TV-MA,TV Comedies


In [33]:
actors = []

for i in movies['cast']:
    actor = re.split(r', \s*', i)
    actors.append(actor)
    
flat_list = []
for sublist in actors:
    for item in sublist:
        flat_list.append(item)
        
actors_list = sorted(set(flat_list))

binary_actors = [[0] * 0 for i in range(len(set(flat_list)))]

for i in movies['cast']:
    k = 0
    for j in actors_list:
        if j in i:
            binary_actors[k].append(1.0)
        else:
            binary_actors[k].append(0.0)
        k+=1
        
binary_actors = pd.DataFrame(binary_actors).transpose()
        
directors = []

for i in movies['director']:
    if pd.notna(i):
        director = re.split(r', \s*', i)
        directors.append(director)
    
flat_list2 = []
for sublist in directors:
    for item in sublist:
        flat_list2.append(item)
        
directors_list = sorted(set(flat_list2))

binary_directors = [[0] * 0 for i in range(len(set(flat_list2)))]

for i in movies['director']:
    k = 0
    for j in directors_list:
        if pd.isna(i):
            binary_directors[k].append(0.0)
        elif j in i:
            binary_directors[k].append(1.0)
        else:
            binary_directors[k].append(0.0)
        k+=1
        
binary_directors = pd.DataFrame(binary_directors).transpose()
        
countries = []

for i in movies['country']:
    country = re.split(r', \s*', i)
    countries.append(country)
    
flat_list3 = []
for sublist in countries:
    for item in sublist:
        flat_list3.append(item)
        
countries_list = sorted(set(flat_list3))

binary_countries = [[0] * 0 for i in range(len(set(flat_list3)))]

for i in movies['country']:
    k = 0
    for j in countries_list:
        if j in i:
            binary_countries[k].append(1.0)
        else:
            binary_countries[k].append(0.0)
        k+=1
        
binary_countries = pd.DataFrame(binary_countries).transpose()

genres = []

for i in movies['listed_in']:
    genre = re.split(r', \s*', i)
    genres.append(genre)
    
flat_list4 = []
for sublist in genres:
    for item in sublist:
        flat_list4.append(item)
        
genres_list = sorted(set(flat_list4))

binary_genres = [[0] * 0 for i in range(len(set(flat_list4)))]

for i in movies['listed_in']:
    k = 0
    for j in genres_list:
        if j in i:
            binary_genres[k].append(1.0)
        else:
            binary_genres[k].append(0.0)
        k+=1
        
binary_genres = pd.DataFrame(binary_genres).transpose()

ratings = []

for i in movies['rating']:
    ratings.append(i)

ratings_list = sorted(set(ratings))

binary_ratings = [[0] * 0 for i in range(len(set(ratings_list)))]

for i in movies['rating']:
    k = 0
    for j in ratings_list:
        if j in i:
            binary_ratings[k].append(1.0)
        else:
            binary_ratings[k].append(0.0)
        k+=1
        
binary_ratings = pd.DataFrame(binary_ratings).transpose()

In [39]:
binary = pd.concat([binary_actors, binary_directors, binary_countries, binary_genres], axis=1,ignore_index=True)
binary

,0,1,2,3,4,5,6,7,8,9,...,26570,26571,26572,26573,26574,26575,26576,26577,26578,26579
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4756,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4757,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4758,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4759,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [40]:
actors2 = []

for i in tv['cast']:
    actor2 = re.split(r', \s*', i)
    actors2.append(actor2)
    
flat_list5 = []
for sublist in actors2:
    for item in sublist:
        flat_list5.append(item)
        
actors_list2 = sorted(set(flat_list5))

binary_actors2 = [[0] * 0 for i in range(len(set(flat_list5)))]

for i in tv['cast']:
    k = 0
    for j in actors_list2:
        if j in i:
            binary_actors2[k].append(1.0)
        else:
            binary_actors2[k].append(0.0)
        k+=1
        
binary_actors2 = pd.DataFrame(binary_actors2).transpose()
        

countries2 = []

for i in tv['country']:
    country2 = re.split(r', \s*', i)
    countries2.append(country2)
    
flat_list6 = []
for sublist in countries2:
    for item in sublist:
        flat_list6.append(item)
        
countries_list2 = sorted(set(flat_list6))

binary_countries2 = [[0] * 0 for i in range(len(set(flat_list6)))]

for i in tv['country']:
    k = 0
    for j in countries_list2:
        if j in i:
            binary_countries2[k].append(1.0)
        else:
            binary_countries2[k].append(0.0)
        k+=1
        
binary_countries2 = pd.DataFrame(binary_countries2).transpose()

genres2 = []

for i in tv['listed_in']:
    genre2 = re.split(r', \s*', i)
    genres2.append(genre2)
    
flat_list7 = []
for sublist in genres2:
    for item in sublist:
        flat_list7.append(item)
        
genres_list2 = sorted(set(flat_list7))

binary_genres2 = [[0] * 0 for i in range(len(set(flat_list7)))]

for i in tv['listed_in']:
    k = 0
    for j in genres_list2:
        if j in i:
            binary_genres2[k].append(1.0)
        else:
            binary_genres2[k].append(0.0)
        k+=1
        
binary_genres2 = pd.DataFrame(binary_genres2).transpose()

ratings2 = []

for i in tv['rating']:
    ratings2.append(i)

ratings_list2 = sorted(set(ratings2))

binary_ratings2 = [[0] * 0 for i in range(len(set(ratings_list2)))]

for i in tv['rating']:
    k = 0
    for j in ratings_list2:
        if j in i:
            binary_ratings2[k].append(1.0)
        else:
            binary_ratings2[k].append(0.0)
        k+=1
        
binary_ratings2 = pd.DataFrame(binary_ratings2).transpose()

In [41]:
binary2 = pd.concat([binary_actors2, binary_countries2, binary_genres2], axis=1, ignore_index=True)
binary2

,0,1,2,3,4,5,6,7,8,9,...,12741,12742,12743,12744,12745,12746,12747,12748,12749,12750
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1886,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1887,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
1888,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1889,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [42]:
def recommender(search):
    cs_list = []
    binary_list = []
    if search in movies['title'].values:
        idx = movies[movies['title'] == search].index.item()
        for i in binary.iloc[idx]:
            binary_list.append(i)
        point1 = np.array(binary_list).reshape(1, -1)
        point1 = [val for sublist in point1 for val in sublist]    
        for j in range(len(movies)):
            binary_list2 = []
            for k in binary.iloc[j]:
                binary_list2.append(k)
            point2 = np.array(binary_list2).reshape(1, -1)
            point2 = [val for sublist in point2 for val in sublist]
            dot_product = np.dot(point1, point2)
            norm_1 = np.linalg.norm(point1)
            norm_2 = np.linalg.norm(point2)
            cos_sim = dot_product / (norm_1 * norm_2)
            cs_list.append(cos_sim)
        movies_copy = movies.copy()
        movies_copy['cos_sim'] = cs_list
        results = movies_copy.sort_values('cos_sim', ascending=False)
        results = results[results['title'] != search]    
        top_results = results.head(5)
        return(top_results)
    elif search in tv['title'].values:
        idx = tv[tv['title'] == search].index.item()
        for i in binary2.iloc[idx]:
            binary_list.append(i)
        point1 = np.array(binary_list).reshape(1, -1)
        point1 = [val for sublist in point1 for val in sublist]
        for j in range(len(tv)):
            binary_list2 = []
            for k in binary2.iloc[j]:
                binary_list2.append(k)
            point2 = np.array(binary_list2).reshape(1, -1)
            point2 = [val for sublist in point2 for val in sublist]
            dot_product = np.dot(point1, point2)
            norm_1 = np.linalg.norm(point1)
            norm_2 = np.linalg.norm(point2)
            cos_sim = dot_product / (norm_1 * norm_2)
            cs_list.append(cos_sim)
        tv_copy = tv.copy()
        tv_copy['cos_sim'] = cs_list
        results = tv_copy.sort_values('cos_sim', ascending=False)
        results = results[results['title'] != search]    
        top_results = results.head(5)
        return(top_results)
    else:
        return("Title not in dataset. Please check spelling.")

## 4.1. Recommending Movies

In [43]:
recommender('The Conjuring')

,title,director,cast,country,rating,listed_in,cos_sim
1868,Insidious,James Wan,"Patrick Wilson, Rose Byrne, Lin Shaye, Ty Simp...","United States, Canada, United Kingdom",PG-13,"Horror Movies, Thrillers",0.388922
968,Creep,Patrick Brice,"Mark Duplass, Patrick Brice",United States,R,"Horror Movies, Independent Movies, Thrillers",0.377964
1844,In the Tall Grass,Vincenzo Natali,"Patrick Wilson, Laysla De Oliveira, Avery Whit...","Canada, United States",TV-MA,"Horror Movies, Thrillers",0.370625
969,Creep 2,Patrick Brice,"Mark Duplass, Desiree Akhavan, Karan Soni",United States,TV-MA,"Horror Movies, Independent Movies, Thrillers",0.356348
1077,Desolation,Sam Patton,"Jaimi Paige, Alyshia Ochse, Toby Nichols, Clau...",United States,TV-MA,"Horror Movies, Thrillers",0.356348


In [44]:
recommender("Child's Play")

,title,director,cast,country,rating,listed_in,cos_sim
975,Cult of Chucky,Don Mancini,"Fiona Dourif, Michael Therriault, Adam Hurtig,...",United States,R,Horror Movies,0.312500
4669,Wildling,Fritz Böhm,"Bel Powley, Brad Dourif, Liv Tyler, Collin Kel...",United States,R,"Horror Movies, Independent Movies, Sci-Fi & Fa...",0.301511
3592,Stephanie,Akiva Goldsman,"Shree Cooks, Frank Grillo, Anna Torv",United States,R,Horror Movies,0.283473
789,Candyman,Bernard Rose,"Virginia Madsen, Tony Todd, Xander Berkeley, K...","United States, United Kingdom",R,"Cult Movies, Horror Movies",0.267261
968,Creep,Patrick Brice,"Mark Duplass, Patrick Brice",United States,R,"Horror Movies, Independent Movies, Thrillers",0.265165


In [45]:
recommender('Charlie and the Chocolate Factory')

,title,director,cast,country,rating,listed_in,cos_sim
3661,Sweeney Todd: The Demon Barber of Fleet Street,Tim Burton,"Johnny Depp, Helena Bonham Carter, Alan Rickma...","United States, United Kingdom",R,"Dramas, Horror Movies, Music & Musicals",0.360041
1346,Figaro Pho,Luke Jurevicius,Luke Jurevicius,United Kingdom,TV-Y7,"Children & Family Movies, Comedies",0.356348
2983,Penelope,Mark Palansky,"Christina Ricci, James McAvoy, Catherine O'Har...","United Kingdom, United States",PG,"Children & Family Movies, Comedies, Romantic M...",0.314970
1557,Goosebumps 2: Haunted Halloween,Ari Sandel,"Jeremy Ray Taylor, Madison Iseman, Caleel Harr...","United States, United Kingdom",PG,"Children & Family Movies, Comedies",0.302614
1767,Hugo,Martin Scorsese,"Ben Kingsley, Sacha Baron Cohen, Asa Butterfie...","United Kingdom, United States, France",PG,"Children & Family Movies, Dramas",0.293972


In [46]:
recommender('Wild Child')

,title,director,cast,country,rating,listed_in,cos_sim
4018,The Kissing Booth,Vince Marcello,"Joey King, Joel Courtney, Jacob Elordi, Molly ...","United Kingdom, United States",TV-14,"Comedies, Romantic Movies",0.353553
2983,Penelope,Mark Palansky,"Christina Ricci, James McAvoy, Catherine O'Har...","United Kingdom, United States",PG,"Children & Family Movies, Comedies, Romantic M...",0.322749
4019,The Kissing Booth 2,Vince Marcello,"Joey King, Joel Courtney, Jacob Elordi, Molly ...","United Kingdom, United States",TV-14,"Comedies, Romantic Movies",0.310087
3959,The Guernsey Literary and Potato Peel Pie Society,Mike Newell,"Lily James, Michiel Huisman, Penelope Wilton, ...","United Kingdom, France, United States",TV-PG,"Dramas, Romantic Movies",0.298807
1714,Holidate,John Whitesell,"Emma Roberts, Luke Bracey, Kristin Chenoweth, ...",United States,TV-MA,"Comedies, Romantic Movies",0.298807


In [47]:
recommender("Dr. Seuss' The Cat in the Hat")

,title,director,cast,country,rating,listed_in,cos_sim
2798,NOVA: Bird Brain,NaN,Craig Sechler,United States,TV-G,"Children & Family Movies, Documentaries",0.372104
3624,Sugar High,Ariel Boles,Hunter March,United States,TV-G,Children & Family Movies,0.372104
4758,Zoom,Peter Hewitt,"Tim Allen, Courteney Cox, Chevy Chase, Kate Ma...",United States,PG,"Children & Family Movies, Comedies",0.370625
4624,What a Girl Wants,Dennie Gordon,"Amanda Bynes, Colin Firth, Kelly Preston, Eile...","United States, United Kingdom",PG,"Children & Family Movies, Comedies",0.370625
3066,Prince of Peoria: A Christmas Moose Miracle,Jon Rosenbaum,"Gavin Lewis, Theodore Barnes, Shelby Simmons, ...",United States,TV-G,"Children & Family Movies, Comedies",0.369800


In [48]:
recommender('Hook')

,title,director,cast,country,rating,listed_in,cos_sim
477,Balto,Simon Wells,"Kevin Bacon, Bob Hoskins, Bridget Fonda, Jim C...",United States,G,"Children & Family Movies, Dramas",0.370625
3624,Sugar High,Ariel Boles,Hunter March,United States,TV-G,Children & Family Movies,0.358569
2798,NOVA: Bird Brain,NaN,Craig Sechler,United States,TV-G,"Children & Family Movies, Documentaries",0.358569
1001,Dance Dreams: Hot Chocolate Nutcracker,Oliver Bokelberg,Debbie Allen,United States,TV-G,"Children & Family Movies, Documentaries",0.327327
562,Benji's Very Own Christmas Story,Joe Camp,"Ron Moody, Patsy Garrett, Cynthia Smith",United States,TV-G,Children & Family Movies,0.303046


In [49]:
recommender('Holidate')

,title,director,cast,country,rating,listed_in,cos_sim
4646,When We First Met,Ari Sandel,"Adam DeVine, Alexandra Daddario, Shelley Henni...",United States,TV-14,"Comedies, Romantic Movies",0.422577
1360,First Impression,Arthur Muhammad,"Lamman Rucker, Lisa Arrindell Anderson, Elise ...",United States,TV-14,"Comedies, Romantic Movies",0.356348
3606,Stranger than Fiction,Marc Forster,"Will Ferrell, Maggie Gyllenhaal, Dustin Hoffma...","United States, United Kingdom",PG-13,"Comedies, Romantic Movies",0.345033
4018,The Kissing Booth,Vince Marcello,"Joey King, Joel Courtney, Jacob Elordi, Molly ...","United Kingdom, United States",TV-14,"Comedies, Romantic Movies",0.338062
3946,The Good Catholic,Paul Shoulberg,"Zachary Spicer, Wrenn Schmidt, Danny Glover, J...",United States,PG-13,"Comedies, Dramas, Romantic Movies",0.338062


## 4.2. Recommending TV shows

In [50]:
recommender('After Life')

,title,director,cast,country,rating,listed_in,cos_sim
468,Extras,NaN,"Ricky Gervais, Stephen Merchant, Ashley Jensen...","United Kingdom, United States",TV-MA,"British TV Shows, TV Comedies",0.526235
1468,The Blue Planet: A Natural History of the Oceans,Alastair Fothergill,David Attenborough,United Kingdom,TV-G,"British TV Shows, Docuseries, International TV...",0.452911
578,Grand Designs,NaN,Kevin McCloud,United Kingdom,TV-14,"British TV Shows, International TV Shows, Real...",0.452911
1007,My Hotter Half,NaN,Melvin Odoom,United Kingdom,TV-PG,"British TV Shows, International TV Shows, Real...",0.452911
62,Ainsley Eats the Streets,NaN,Ainsley Harriott,United Kingdom,TV-PG,"British TV Shows, Docuseries, International TV...",0.452911


In [51]:
recommender('Anne with an E')

,title,director,cast,country,rating,listed_in,cos_sim
254,Can You Hear Me?,NaN,"Mélissa Bédard, Ève Landry, Florence Longpré",Canada,TV-MA,"International TV Shows, TV Comedies, TV Dramas",0.462250
1229,Restaurants on the Edge,NaN,"Nick Liberato, Karin Bohn, Dennis Prescott",Canada,TV-14,"International TV Shows, Reality TV",0.392232
185,Bitten,NaN,"Laura Vandervoort, Greyston Holt, Greg Bryk, S...",Canada,TV-MA,"International TV Shows, TV Dramas, TV Horror",0.384615
622,Heavy Rescue: 401,NaN,Dave Pettitt,Canada,TV-MA,"International TV Shows, Reality TV",0.372104
645,Hip-Hop Evolution,NaN,Shad Kabango,Canada,TV-MA,"Docuseries, International TV Shows",0.372104


In [52]:
recommender('Gilmore Girls')

,title,director,cast,country,rating,listed_in,cos_sim
542,Gilmore Girls: A Year in the Life,NaN,"Lauren Graham, Alexis Bledel, Kelly Bishop, Sc...",United States,TV-14,"TV Comedies, TV Dramas, Teen TV Shows",0.777778
863,Love,NaN,"Gillian Jacobs, Paul Rust, Claudia O'Doherty",United States,TV-MA,"Romantic TV Shows, TV Comedies, TV Dramas",0.416667
1061,No Tomorrow,NaN,"Joshua Sasse, Tori Anderson, Jonathan Langdon,...",United States,TV-PG,"Romantic TV Shows, TV Comedies, TV Dramas",0.408248
361,DASH & LILY,NaN,"Midori Francis, Austin Abrams, Dante Brown, Tr...",United States,TV-14,"Romantic TV Shows, TV Comedies, TV Dramas",0.392837
664,Hot Date,NaN,"Emily Axford, Brian Murphy",United States,TV-MA,"Romantic TV Shows, TV Comedies",0.384900


In [53]:
recommender('Friends')

,title,director,cast,country,rating,listed_in,cos_sim
455,Episodes,NaN,"Matt LeBlanc, Tamsin Greig, Stephen Mangan, Ka...","United Kingdom, United States",TV-MA,"Classic & Cult TV, TV Comedies",0.476731
1133,Pee-wee's Playhouse,NaN,Paul Reubens,United States,TV-PG,"Classic & Cult TV, Kids' TV, TV Comedies",0.424264
513,Frasier,NaN,"Kelsey Grammer, Jane Leeves, David Hyde Pierce...",United States,TV-PG,"Classic & Cult TV, TV Comedies",0.400000
910,Man with a Plan,NaN,"Matt LeBlanc, Liza Snyder, Grace Kaufman, Matt...",United States,TV-PG,TV Comedies,0.400000
1550,The Honeymoon Stand Up Special,NaN,"Natasha Leggero, Moshe Kasher",United States,TV-MA,"Stand-Up Comedy & Talk Shows, TV Comedies",0.387298


In [54]:
recommender('Love on the Spectrum')

,title,director,cast,country,rating,listed_in,cos_sim
278,Cheapest Weddings,NaN,Melanie Vallejo,Australia,TV-14,"International TV Shows, Reality TV",0.730297
1890,Zumbo's Just Desserts,NaN,"Adriano Zumbo, Rachel Khoo",Australia,TV-PG,"International TV Shows, Reality TV",0.666667
1782,Vai Anitta,NaN,Anitta,Brazil,TV-MA,"Docuseries, International TV Shows, Reality TV",0.666667
326,Court Justice,NaN,Chris Bourke,Australia,TV-MA,"Crime TV Shows, Docuseries, International TV S...",0.666667
21,72 Dangerous Animals: Asia,NaN,Bob Brisbane,Australia,TV-14,"Docuseries, International TV Shows, Science & ...",0.666667


In [55]:
recommender('13 Reasons Why')

,title,director,cast,country,rating,listed_in,cos_sim
11,13 Reasons Why: Beyond the Reasons,NaN,"Dylan Minnette, Katherine Langford, Kate Walsh...",United States,TV-MA,"Crime TV Shows, Docuseries",0.432789
515,Frequency,NaN,"Peyton List, Riley Smith, Mekhi Phifer, Devin ...",United States,TV-14,"Crime TV Shows, TV Dramas, TV Mysteries",0.381771
1653,The Sinner,NaN,"Jessica Biel, Bill Pullman, Christopher Abbott...",United States,TV-MA,"Crime TV Shows, TV Dramas, TV Mysteries",0.362738
964,MINDHUNTER,NaN,"Jonathan Groff, Holt McCallany, Anna Torv, Cot...",United States,TV-MA,"Crime TV Shows, TV Dramas, TV Mysteries",0.362738
1775,Unsolved,NaN,"Josh Duhamel, Jimmi Simpson, Bokeem Woodbine",United States,TV-MA,"Crime TV Shows, TV Dramas",0.346844


In [56]:
recommender('Derry Girls')

,title,director,cast,country,rating,listed_in,cos_sim
332,Crazyhead,NaN,"Cara Theobold, Susan Wokoma, Riann Steele, Ari...",United Kingdom,TV-MA,"British TV Shows, International TV Shows, TV C...",0.420084
852,Loaded,NaN,"Jim Howick, Samuel Anderson, Jonny Sweet, Nick...",United Kingdom,TV-MA,"British TV Shows, International TV Shows, TV C...",0.403604
502,Flowers,NaN,"Olivia Colman, Julian Barratt, Daniel Rigby, S...",United Kingdom,TV-MA,"British TV Shows, International TV Shows, TV C...",0.403604
180,"Big Dreams, Small Spaces",NaN,Monty Don,United Kingdom,TV-G,"British TV Shows, International TV Shows, Real...",0.396059
985,Monty Don's Italian Gardens,NaN,Monty Don,United Kingdom,TV-G,"British TV Shows, Docuseries, International TV...",0.396059


In [57]:
recommender('Breaking Bad')

,title,director,cast,country,rating,listed_in,cos_sim
169,Better Call Saul,NaN,"Bob Odenkirk, Jonathan Banks, Michael McKean, ...",United States,TV-MA,"Crime TV Shows, TV Comedies, TV Dramas",0.521749
1459,The Assassination of Gianni Versace,NaN,"Edgar Ramírez, Darren Criss, Ricky Martin, Pen...",United States,TV-MA,"Crime TV Shows, TV Dramas, TV Thrillers",0.430331
1775,Unsolved,NaN,"Josh Duhamel, Jimmi Simpson, Bokeem Woodbine",United States,TV-MA,"Crime TV Shows, TV Dramas",0.390360
537,Get Shorty,NaN,"Ray Romano, Chris O'Dowd",United States,TV-MA,"Crime TV Shows, TV Comedies, TV Dramas",0.390360
1416,Surviving R. Kelly,NaN,R. Kelly,United States,TV-MA,"Crime TV Shows, Docuseries",0.346410


In [58]:
recommender('Stranger Things')

,title,director,cast,country,rating,listed_in,cos_sim
175,Beyond Stranger Things,NaN,"Jim Rash, Matt Duffer, Ross Duffer, Shawn Levy...",United States,TV-14,"Stand-Up Comedy & Talk Shows, TV Mysteries, TV...",0.741941
1174,Prank Encounters,NaN,Gaten Matarazzo,United States,TV-MA,"Reality TV, TV Comedies, TV Horror",0.292770
625,Helix,NaN,"Billy Campbell, Hiroyuki Sanada, Kyra Zagorsky...","United States, Canada",TV-MA,"TV Horror, TV Mysteries, TV Sci-Fi & Fantasy",0.264628
1811,Warrior Nun,NaN,"Alba Baptista, Toya Turner, Lorena Andrea, Kri...",United States,TV-MA,"TV Action & Adventure, TV Mysteries, TV Sci-Fi...",0.251976
292,Chilling Adventures of Sabrina,NaN,"Kiernan Shipka, Ross Lynch, Miranda Otto, Lucy...",United States,TV-14,"TV Horror, TV Mysteries, TV Sci-Fi & Fantasy",0.250313


# 5. Developing Recommendation Engine using *Movie/TV show descriptions*

In [59]:
movies_des = data[data['type'] == 'Movie'].reset_index()
movies_des = movies_des[['title', 'description']]
movies_des.head()

,title,description
0,7:19,After a devastating earthquake hits Mexico Cit...
1,23:59,"When an army recruit is found dead, his fellow..."
2,9,"In a postapocalyptic world, rag-doll robots hi..."
3,21,A brilliant group of students become card-coun...
4,122,"After an awful accident, a couple admitted to ..."


In [60]:
tv_des = data[data['type'] == 'TV Show'].reset_index()
tv_des = tv_des[['title', 'description']]
tv_des.head()

,title,description
0,3%,In a future where the elite inhabit an island ...
1,46,A genetics professor experiments with a treatm...
2,1983,"In this dark alt-history thriller, a naïve law..."
3,​SAINT SEIYA: Knights of the Zodiac,Seiya and the Knights of the Zodiac rise again...
4,#blackAF,Kenya Barris and his family navigate relations...


In [61]:
filtered_movies = []
movies_words = []

for text in movies_des['description']:
    text_tokens = word_tokenize(text)
    tokens_without_sw = [word.lower() for word in text_tokens if not word in stopwords.words()]
    movies_words.append(tokens_without_sw)
    filtered = (" ").join(tokens_without_sw)
    filtered_movies.append(filtered)

movies_words = [val for sublist in movies_words for val in sublist]
movies_words = sorted(set(movies_words))
movies_des['description_filtered'] = filtered_movies
movies_des.head()

,title,description,description_filtered
0,7:19,After a devastating earthquake hits Mexico Cit...,after devastating earthquake hits mexico city ...
1,23:59,"When an army recruit is found dead, his fellow...","when army recruit found dead , fellow soldiers..."
2,9,"In a postapocalyptic world, rag-doll robots hi...","in postapocalyptic world , rag-doll robots hid..."
3,21,A brilliant group of students become card-coun...,a brilliant group students become card-countin...
4,122,"After an awful accident, a couple admitted to ...","after awful accident , couple admitted grisly ..."


In [62]:
filtered_tv = []
tv_words = []

for text in tv_des['description']:
    text_tokens = word_tokenize(text)
    tokens_without_sw = [word.lower() for word in text_tokens if not word in stopwords.words()]
    tv_words.append(tokens_without_sw)
    filtered = (" ").join(tokens_without_sw)
    filtered_tv.append(filtered)

tv_words = [val for sublist in tv_words for val in sublist]
tv_words = sorted(set(tv_words))
tv_des['description_filtered'] = filtered_tv
tv_des.head()

,title,description,description_filtered
0,3%,In a future where the elite inhabit an island ...,in future elite inhabit island paradise far cr...
1,46,A genetics professor experiments with a treatm...,a genetics professor experiments treatment com...
2,1983,"In this dark alt-history thriller, a naïve law...","in dark alt-history thriller , naïve law stude..."
3,​SAINT SEIYA: Knights of the Zodiac,Seiya and the Knights of the Zodiac rise again...,seiya knights zodiac rise protect reincarnatio...
4,#blackAF,Kenya Barris and his family navigate relations...,"kenya barris family navigate relationships , r..."


In [63]:
movie_word_binary = [[0] * 0 for i in range(len(set(movies_words)))]

for des in movies_des['description_filtered']:
    k = 0
    for word in movies_words:
        if word in des:
            movie_word_binary[k].append(1.0)
        else:
            movie_word_binary[k].append(0.0)
        k+=1
        
movie_word_binary = pd.DataFrame(movie_word_binary).transpose()

In [64]:
tv_word_binary = [[0] * 0 for i in range(len(set(tv_words)))]

for des in tv_des['description_filtered']:
    k = 0
    for word in tv_words:
        if word in des:
            tv_word_binary[k].append(1.0)
        else:
            tv_word_binary[k].append(0.0)
        k+=1
        
tv_word_binary = pd.DataFrame(tv_word_binary).transpose()

In [65]:
def recommender2(search):
    cs_list = []
    binary_list = []
    if search in movies_des['title'].values:
        idx = movies_des[movies_des['title'] == search].index.item()
        for i in movie_word_binary.iloc[idx]:
            binary_list.append(i)
        point1 = np.array(binary_list).reshape(1, -1)
        point1 = [val for sublist in point1 for val in sublist]    
        for j in range(len(movies_des)):
            binary_list2 = []
            for k in movie_word_binary.iloc[j]:
                binary_list2.append(k)
            point2 = np.array(binary_list2).reshape(1, -1)
            point2 = [val for sublist in point2 for val in sublist]
            dot_product = np.dot(point1, point2)
            norm_1 = np.linalg.norm(point1)
            norm_2 = np.linalg.norm(point2)
            cos_sim = dot_product / (norm_1 * norm_2)
            cs_list.append(cos_sim)
        movies_copy = movies_des.copy()
        movies_copy['cos_sim'] = cs_list
        results = movies_copy.sort_values('cos_sim', ascending=False)
        results = results[results['title'] != search]    
        top_results = results.head(5)
        return(top_results)
    elif search in tv_des['title'].values:
        idx = tv_des[tv_des['title'] == search].index.item()
        for i in tv_word_binary.iloc[idx]:
            binary_list.append(i)
        point1 = np.array(binary_list).reshape(1, -1)
        point1 = [val for sublist in point1 for val in sublist]
        for j in range(len(tv)):
            binary_list2 = []
            for k in tv_word_binary.iloc[j]:
                binary_list2.append(k)
            point2 = np.array(binary_list2).reshape(1, -1)
            point2 = [val for sublist in point2 for val in sublist]
            dot_product = np.dot(point1, point2)
            norm_1 = np.linalg.norm(point1)
            norm_2 = np.linalg.norm(point2)
            cos_sim = dot_product / (norm_1 * norm_2)
            cs_list.append(cos_sim)
        tv_copy = tv_des.copy()
        tv_copy['cos_sim'] = cs_list
        results = tv_copy.sort_values('cos_sim', ascending=False)
        results = results[results['title'] != search]    
        top_results = results.head(5)
        return(top_results)
    else:
        return("Title not in dataset. Please check spelling.")

## 5.1. Recommending Movies

In [66]:
pd.options.display.max_colwidth = 300
recommender2('The Conjuring')

,title,description,description_filtered,cos_sim
1632,Hard Lessons,"This drama based on real-life events tells the story of George McKenna, the tough, determined new principal of a notorious Los Angeles high school.","this drama based real-life events tells story george mckenna , tough , determined new principal notorious los angeles high school .",0.489419
3335,Sat Sri Akal,"Based on true events, this moving story centers on a Punjabi family whose celebration of their faith endures in the face of conflicting attitudes.","based true events , moving story centers punjabi family whose celebration faith endures conflicting attitudes .",0.478650
2549,Mirai,"Unhappy after his new baby sister displaces him, four-year-old Kun begins meeting people and pets from his family's history in their unique house.","unhappy new baby sister displaces , four-year-old kun begins meeting people pets family 's history unique house .",0.478345
3910,The Eyes of My Mother,"At the remote farmhouse where she once witnessed a traumatic childhood event, a young woman develops a grisly fascination with violence.","at remote farmhouse witnessed traumatic childhood event , young woman develops grisly fascination violence .",0.470605
3578,Standoff,"After witnessing an assassin's slaughter, a young girl holes up in a farmhouse with a suicidal vet, who must use wits and guts to fend off the killer.","after witnessing assassin 's slaughter , young girl holes farmhouse suicidal vet , must use wits guts fend killer .",0.460628


In [67]:
recommender2("Child's Play")

,title,description,description_filtered,cos_sim
2467,Material,"A dutiful son must hide his pursuit of stand-up comedy from his staunch father, who expects him to inherit his store and uphold their Muslim beliefs.","a dutiful must hide pursuit stand-up comedy staunch father , expects inherit store uphold muslim beliefs .",0.431291
3605,Strange Weather,"A grieving mother takes a road trip with her friend to confront the man who, she believes, stole her late son's business idea to get rich.","a grieving mother takes road trip friend confront , believes , stole late 's business idea get rich .",0.420183
3553,Spectral,"When an otherworldly force wreaks havoc on a war-torn European city, an engineer teams up with an elite Special Ops unit to stop it.","when otherworldly force wreaks havoc war-torn european city , engineer teams elite special ops unit stop .",0.414039
1550,Good People,A struggling couple can't believe their luck when they find a stash of money in the apartment of a neighbor who was recently murdered.,a struggling couple n't believe luck find stash money apartment neighbor recently murdered .,0.411766
1017,Dark Places,"Years after surviving a brutal crime as a child, Libby Day comes to believe that the brother she testified against for committing it may be innocent.","years surviving brutal crime child , libby day comes believe brother testified committing may innocent .",0.410391


In [68]:
recommender2('Charlie and the Chocolate Factory')

,title,description,description_filtered,cos_sim
4671,Willy Wonka & the Chocolate Factory,Zany Willy Wonka causes a stir when he announces that golden tickets hidden inside his candy bars will admit holders into his secret confectionary.,zany willy wonka causes stir announces golden tickets hidden inside candy bars admit holders secret confectionary .,0.400892
2040,Kaake Da Viyah,"In this zany comedy, a man is torn between the girl he loves and the respective women his warring mother and grandmother have chosen for him to marry.","in zany comedy , torn girl loves respective women warring mother grandmother chosen marry .",0.383326
3796,The Boss's Daughter,"While working together, a married textile foreman and his boss’s daughter have a torrid love affair, stirring up hostility among the factory crew.","while working together , married textile foreman boss ’ daughter torrid love affair , stirring hostility among factory crew .",0.381000
4306,The Wishing Tree,Five children living on the edge of a forest band together to save a magical wishing tree from destruction and find their own paths to happiness.,five children living edge forest band together save magical wishing tree destruction find paths happiness .,0.368856
1884,Inxeba,"At an initiation ritual for the young men of his Xhosa community, a closeted factory hand meets a Johannesburg teen who discovers his best-kept secret.","at initiation ritual young xhosa community , closeted factory hand meets johannesburg teen discovers best-kept secret .",0.363803


In [69]:
recommender2('Wild Child')

,title,description,description_filtered,cos_sim
1319,Fanatyk,"As a son deals with his own struggles, he must calm his father's obsession with fishing before his outlandish behavior ruins the entire family.","as deals struggles , must calm father 's obsession fishing outlandish behavior ruins entire family .",0.497940
1217,Either Me Or My Auntie,"A musician's marriage proposal to his girlfriend is denied by her mother, whose affinity for magic begins to meddle in their relationship even more.","a musician 's marriage proposal girlfriend denied mother , whose affinity magic begins meddle relationship even .",0.468979
2694,My Own Man,"When a man discovers he will be the father to a boy, his fear and insecurities send him on an emotional, humorous quest for his own manhood.","when discovers father boy , fear insecurities send emotional , humorous quest manhood .",0.453565
2554,Misfit,"After living in America for years, a teenage girl moves back to the Netherlands and is quickly singled out as a misfit by the popular clique at school.","after living america years , teenage girl moves back netherlands quickly singled misfit popular clique school .",0.448014
1288,Evvarikee Cheppoddu,"When caste differences throw a wrench into their otherwise blossoming relationship, a couple must somehow convince the girl’s father to let them marry.","when caste differences throw wrench otherwise blossoming relationship , couple must somehow convince girl ’ father let marry .",0.446630


In [70]:
recommender2("Dr. Seuss' The Cat in the Hat")

,title,description,description_filtered,cos_sim
1660,He's Out There,"While vacationing at a remote lake house, a mother and her daughters become pawns in the twisted game of an ax-wielding psychopath.","while vacationing remote lake house , mother daughters become pawns twisted game ax-wielding psychopath .",0.486506
1016,Dark Light,"Implicated in her daughter's disappearance, a mother searches for answers when she returns to an old family home that may have an unwanted visitor.","implicated daughter 's disappearance , mother searches answers returns old family home may unwanted visitor .",0.483046
1853,INDIA,"A man buys a young girl, code-names her ""Doll"" and sends her to live with a mother of two who has a mysterious mission to be fulfilled.","a buys young girl , code-names `` doll '' sends live mother two mysterious mission fulfilled .",0.460195
3808,The Breadwinner,A courageous 11-year-old Afghan girl disguises herself as a boy and takes on odd jobs to provide for her family when her father is arrested.,a courageous 11-year-old afghan girl disguises boy takes odd jobs provide family father arrested .,0.454621
1217,Either Me Or My Auntie,"A musician's marriage proposal to his girlfriend is denied by her mother, whose affinity for magic begins to meddle in their relationship even more.","a musician 's marriage proposal girlfriend denied mother , whose affinity magic begins meddle relationship even .",0.444117


In [71]:
recommender2('Hook')

,title,description,description_filtered,cos_sim
4371,Tinker Bell and the Legend of the NeverBeast,"When suspicious scout fairies scheme to capture the fearsome-looking NeverBeast, Tinker Bell and Fawn set out to save the wondrous creature.","when suspicious scout fairies scheme capture fearsome-looking neverbeast , tinker bell fawn set save wondrous creature .",0.425195
2113,Kickboxer: Retaliation,"Sloan's vow to never return to Thailand is cut short when he's kidnapped and taken to a Thai prison, where he's forced to fight a 400-pound brute.","sloan 's vow never return thailand cut short 's kidnapped taken thai prison , 's forced fight 400-pound brute .",0.418167
1740,Hostiles,"After a long career battling the Cheyenne, a U.S. Army captain is ordered to safely escort the tribe's most influential chief to his Montana homeland.","after long career battling cheyenne , u.s. army captain ordered safely escort tribe 's influential chief montana homeland .",0.415629
1515,Girlfriend's Day,"When he's caught up in a deadly conspiracy, an unemployed greeting card writer must create the perfect card for a new holiday to save his skin.","when 's caught deadly conspiracy , unemployed greeting card writer must create perfect card new holiday save skin .",0.415168
2804,NOVA: Decoding the Weather Machine,"Scientists investigate Earth’s climate machine, looking for clues across the globe, from Greenland’s ice sheet to the desert of Australia.","scientists investigate earth ’ climate machine , looking clues across globe , greenland ’ ice sheet desert australia .",0.414781


In [72]:
recommender2('Holidate')

,title,description,description_filtered,cos_sim
4707,Yanda Kartavya Aahe,"Thanks to an arranged marriage that was designed to make an ailing grandmother happy, newlyweds Rahul and Swati are virtually strangers. Can a four-day honeymoon make a difference when it comes to the couple's feelings about each other?","thanks arranged marriage designed make ailing grandmother happy , newlyweds rahul swati virtually strangers . can four-day honeymoon make difference comes couple 's feelings ?",0.449089
3165,Red Christmas,A family Christmas gathering at an isolated Australian estate turns into the holiday from hell after a mysterious stranger turns up at the door.,a family christmas gathering isolated australian estate turns holiday hell mysterious stranger turns .,0.446663
4527,Us and Them,"Two strangers meet on a train and form a bond that evolves over the years. After a separation, they reconnect and reflect on their love for each other.","two strangers meet train form bond evolves years . after separation , reconnect reflect love .",0.445435
286,All of You,"Two strangers meet on a dating app and experience instant chemistry, but their relationship unravels as jarring differences catch up to them.","two strangers meet dating app experience instant chemistry , relationship unravels jarring differences catch .",0.439155
3963,The Hateful Eight,"Years after the Civil War, a bounty hunter and his captive are waylaid by a Wyoming blizzard and hole up in a way station with six dicey strangers.","years civil war , bounty hunter captive waylaid wyoming blizzard hole way station six dicey strangers .",0.439155


## 5.2. Recommending TV shows

In [73]:
recommender2('After Life')

,title,description,description_filtered,cos_sim
1628,The Paper,A construction magnate takes over a struggling newspaper and attempts to wield editorial influence for power and personal gain.,a construction magnate takes struggling newspaper attempts wield editorial influence power personal gain .,0.483546
1821,Welcome to the Family,"When an evicted single mom's estranged father dies, she and his second wife cover up his death after learning they've been written out of his will.","when evicted single mom 's estranged father , second wife cover death learning 've written .",0.455016
534,Gentlemen and Gangsters,"Now on the run, a writer relates his previous year's escapades when he got sucked into the thrilling, sordid orbit of boxer and jazz man Henry Morgan.","now run , writer relates previous year 's escapades got sucked thrilling , sordid orbit boxer jazz henry morgan .",0.437880
1848,Winter Sun,"Years after ruthless businessmen kill his father and order the death of his twin brother, a modest fisherman adopts a new persona to exact revenge.","years ruthless businessmen kill father order death twin brother , modest fisherman adopts new persona exact revenge .",0.437384
858,Longmire,This contemporary crime thriller focuses on a Wyoming sheriff who's rebuilding his life and career following the death of his wife.,this contemporary crime thriller focuses wyoming sheriff 's rebuilding life career following death wife .,0.425307


In [74]:
recommender2('Anne with an E')

,title,description,description_filtered,cos_sim
764,Khaani,"After a rich politician's son kills a young woman's brother, an unlikely romantic connection complicates her pursuit of justice.","after rich politician 's kills young woman 's brother , unlikely romantic connection complicates pursuit justice .",0.410112
348,Dad's Army,This beloved sitcom follows the unlikely heroes of the volunteer British Home Guard as they prepare for German invasion in World War II.,this beloved sitcom follows unlikely heroes volunteer british home guard prepare german invasion world war ii .,0.405726
147,Bat Pat,"A curious and talkative bat finds spooky fun on the streets of Fogville, a town that's not as quiet as it seems, with a plucky girl and her brothers.","a curious talkative bat finds spooky fun streets fogville , town 's quiet seems , plucky girl brothers .",0.397360
519,From Dusk Till Dawn,Bank-robbing brothers encounter vengeful lawmen and demons south of the border in this original series based on Robert Rodriguez' cult horror film.,bank-robbing brothers encounter vengeful lawmen demons south border original series based robert rodriguez ' cult horror film .,0.392920
370,DC's Legends of Tomorrow,"A mysterious ""time master"" from the future unites an unlikely group of superheroes and villains to save the world from a powerful evil.",a mysterious `` time master '' future unites unlikely group superheroes villains save world powerful evil .,0.389490


In [75]:
recommender2('Gilmore Girls')

,title,description,description_filtered,cos_sim
1871,YooHoo to the Rescue,"In a series of magical missions, quick-witted YooHoo and his can-do crew travel the globe to help animals in need.","in series magical missions , quick-witted yoohoo can-do crew travel globe help animals need .",0.419640
816,Legacies,"Born into a rare supernatural bloodline, Hope Mikaelson attends a gifted private school to master her powers and control her innate urges for evil.","born rare supernatural bloodline , hope mikaelson attends gifted private school master powers control innate urges evil .",0.392232
1290,Seis Manos,"Orphans raised by a martial arts master are plunged into a mystery involving demonic powers, drug cartels, ancient rituals and blood sacrifice.","orphans raised martial arts master plunged mystery involving demonic powers , cartels , ancient rituals blood sacrifice .",0.387836
1510,The Duchess,"Katherine's a single mom juggling her career, her tween daughter, her relationship with her boyfriend — and pondering getting pregnant with her ex.","katherine 's single mom juggling career , tween daughter , relationship boyfriend — pondering getting pregnant ex .",0.376256
887,Luis Miguel - The Series,"This series dramatizes the life story of Mexican superstar singer Luis Miguel, who has captivated audiences in Latin America and beyond for decades.","this series dramatizes life story mexican superstar singer luis miguel , captivated audiences latin america beyond decades .",0.376256


In [76]:
recommender2('Friends')

,title,description,description_filtered,cos_sim
1700,Thomas and Friends,"This animated children's series follows the adventures of Thomas, a cheerful blue tank engine who lives on the island of Sodor.","this animated children 's series follows adventures thomas , cheerful blue tank engine lives island sodor .",0.419292
211,Bondi Rescue,"This reality series follows elite professional lifeguards on Sydney's Bondi Beach, as they take on everything from animal stings to criminals.","this reality series follows elite professional lifeguards sydney 's bondi beach , everything animal stings criminals .",0.403376
593,H,"At a dysfunctional hospital in Paris, three bumbling, eccentric medical employees embark on zany misadventures with surgical imprecision.","at dysfunctional hospital paris , three bumbling , eccentric medical employees embark zany misadventures surgical imprecision .",0.403101
1266,Saint Seiya: The Lost Canvas,This anime adventure follows the battle between a saint of Athena and an avatar of Hades who's working on a painting that could destroy the world.,this anime adventure follows battle saint athena avatar hades 's working painting could destroy world .,0.401772
131,Bad Blood,"This sprawling crime drama follows the true story of the Rizzuto family and its associates, who presided over organized crime in Montreal for decades.","this sprawling crime drama follows true story rizzuto family associates , presided organized crime montreal decades .",0.398790


In [77]:
recommender2('Love on the Spectrum')

,title,description,description_filtered,cos_sim
1762,Ultramarine Magmell,"Decades after the sudden birth of a new continent, a young rescuer-for-hire provides aid to adventurers exploring this dangerous, uncharted world.","decades sudden birth new continent , young rescuer-for-hire provides aid adventurers exploring dangerous , uncharted world .",0.442339
1759,Ugly Duckling,"Young women face up to their insecurities and circumstances, finding love and laughs along the way. Featuring a different storyline every season.","young women insecurities circumstances , finding love laughs along way . featuring different storyline every season .",0.437374
7,100 Humans,"One hundred hardy souls from diverse backgrounds participate in playful experiments exploring age, sex, happiness and other aspects of being human.","one hundred hardy souls diverse backgrounds participate playful experiments exploring age , sex , happiness aspects human .",0.433148
857,London Spy,"When his reclusive-banker lover disappears, a hard-partying young British hedonist plunges into the dangerous world of espionage to find the truth.","when reclusive-banker lover disappears , hard-partying young british hedonist plunges dangerous world espionage find truth .",0.423900
426,Durarara!!,"A young man looking for excitement moves to Ikebukuro, a district in Tokyo, where he finds a world of mystery in which the past and present mesh.","a young looking excitement moves ikebukuro , district tokyo , finds world mystery past present mesh .",0.416463


In [78]:
recommender2('13 Reasons Why')

,title,description,description_filtered,cos_sim
882,"Love, Chunibyo & Other Delusions!","High schooler Yuta wants to leave his past as a delusional weirdo behind. But his classmate Rikka has her own delusions, and she's interested in his.","high schooler yuta wants leave past delusional weirdo behind . but classmate rikka delusions , 's interested .",0.472192
467,Extracurricular,A model high school student who's steeped in a world of serious crime finds his double life upended when a classmate takes an interest in his secret.,a model high school student 's steeped world serious crime finds double life upended classmate takes interest secret .,0.424973
126,Baby,"Fed up with their families and classmates, two teen girls from a wealthy part of Rome are drawn to the city's underworld and start leading double lives.","fed families classmates , two teen girls wealthy part rome drawn city 's underworld start leading double lives .",0.424190
297,CLANNAD,Maladjusted high school student Tomoya's life begins to change when he befriends his classmate Nagisa and several other eccentric girls.,maladjusted high school student tomoya 's life begins change befriends classmate nagisa several eccentric girls .,0.406562
1095,One Day at a Time,"In a reimagining of the TV classic, a newly single Latina mother raises her teen daughter and tween son with the ""help"" of her old-school mom.","in reimagining tv classic , newly single latina mother raises teen daughter tween `` help '' old-school mom .",0.405798


In [79]:
recommender2('Derry Girls')

,title,description,description_filtered,cos_sim
1258,Rurouni Kenshin,"A nomadic swordsman arrives at a martial arts school in Meiji-era Japan, where he lands in the middle of a conflict involving the opium trade.","a nomadic swordsman arrives martial arts school meiji-era japan , lands middle conflict involving opium trade .",0.449171
1631,The Politician,Rich kid Payton has always known he's going to be president. But first he has to navigate the most treacherous political landscape of all: high school.,rich kid payton always known 's going president . but first navigate treacherous political landscape : high school .,0.437361
1867,Yeh Meri Family,"In the summer of 1998, middle child Harshu balances school, family, friendship and other challenges of growing up.","in summer 1998 , middle child harshu balances school , family , friendship challenges growing .",0.425046
1101,One-Punch Man,"The most powerful superhero in the world can kill anyone with one blow. But nothing can challenge him, so he struggles with ennui and depression.","the powerful superhero world kill anyone blow . but nothing challenge , struggles ennui depression .",0.419330
940,Medici: Masters of Florence,"After his father's murder, banking heir Cosimo Medici battles opponents of his artistic, economic and political visions for 15th-century Florence.","after father 's murder , banking heir cosimo medici battles opponents artistic , economic political visions 15th-century florence .",0.417407


In [80]:
recommender2('Breaking Bad')

,title,description,description_filtered,cos_sim
996,Mr. Iglesias,Hilarious high school teacher Gabriel Iglesias tries to make a difference in the lives of some smart but underperforming students at his alma mater.,hilarious high school teacher gabriel iglesias tries make difference lives smart underperforming students alma mater .,0.467099
931,Marvel's The Punisher,A former Marine out to punish the criminals responsible for his family's murder finds himself ensnared in a military conspiracy.,a former marine punish criminals responsible family 's murder finds ensnared military conspiracy .,0.456435
1297,Servant of the People,"After a Ukrainian high school teacher's tirade against government corruption goes viral on social media, he finds himself the country's new president.","after ukrainian high school teacher 's tirade government corruption goes viral social media , finds country 's new president .",0.453632
443,El Reemplazante,A former high-ranking financial executive finds redemption and romance when he's paroled after a prison sentence and becomes a math teacher.,a former high-ranking financial executive finds redemption romance 's paroled prison sentence becomes math teacher .,0.443622
132,Bad Education,"A history teacher at the posh Abbey Grove, Alfie Wickers is something truly special: He's his school's, if not England's, single worst educator.","a history teacher posh abbey grove , alfie wickers something truly special : he 's school 's , england 's , single worst educator .",0.412479


In [81]:
recommender2('Stranger Things')

,title,description,description_filtered,cos_sim
727,Jamtara - Sabka Number Ayega,"A group of small-town young men run a lucrative phishing operation, until a corrupt politician wants in on their scheme – and a cop wants to fight it.","a group small-town young run lucrative phishing operation , corrupt politician wants scheme – cop wants fight .",0.469790
186,Bitter Daisies,"While investigating the disappearance of a teen girl in a tight-knit Galician town, a Civil Guard officer uncovers secrets linked to a loss of her own.","while investigating disappearance teen girl tight-knit galician town , civil guard officer uncovers secrets linked loss .",0.455468
1121,Paradise PD,"An eager young rookie joins the ragtag small-town police force led by his dad as they bumble, squabble and snort their way through a big drug case.","an eager young rookie joins ragtag small-town police force led dad bumble , squabble snort way big case .",0.443310
1267,Sakho & Mangane,A by-the-book police captain and a brash young detective must team up to take on the supernatural when strange forces begin to wreak havoc on Dakar.,a by-the-book police captain brash young detective must team supernatural strange forces begin wreak havoc dakar .,0.435665
1650,The Search,"When a girl vanishes from a suburb near Mexico City, the personal goals of some involved in the case muddy the search. Based on a true story.","when girl vanishes suburb near mexico city , personal goals involved case muddy search . based true story .",0.428929


# 6. Conclusion

Taking the cast, director, country, rating and genres as features rather than the descriptions was definitely the better option. Some of the recommendations by descriptions are good such as the 'Tinker Bell and the Legend of the NeverBeast' recommendation for 'Hook' and the 'Extracurricular' recommendation for '13 Reasons Why', but most of them are from completely different genres with very little in common besides a few key words.